# Setup

In [200]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [201]:
import numpy as np
import os
import pandas as pd
import plotly.express as px

from cost import *
from plotting import *
from prices import *
from imputation import *
from inflation import *
from regression import *
from utils import *

In [202]:
results_dir = 'results/amortized-hardware-costs-finetune-fix/'
os.makedirs(results_dir, exist_ok=True)

In [203]:
cost_estimation_function = estimate_amortized_hardware_costs
compute_percentile_threshold = 50  # select systems >= this percentile of training compute

# Load data

In [204]:
frontier_pcd_df, hardware_df, price_df = load_data_for_cost_estimation(
    compute_percentile_threshold=compute_percentile_threshold
)

In [205]:
len(frontier_pcd_df), len(hardware_df), len(price_df)

(148, 5509, 94)

# Cost estimation

In [206]:
cost_df = cost_estimation_function(frontier_pcd_df, hardware_df, price_df)

==== System: Gemini Ultra ====
Trying Google TPU v4
Could not find hardware model for Google TPU v4


==== System: Qwen-72B ====
Could not find hardware model for Qwen-72B


==== System: Inflection-2 ====
Trying NVIDIA H100 SXM5
Found price: 39546.976

==== System: Nemotron-3-8B ====
Trying NVIDIA A100
Soft matching NVIDIA A100 to NVIDIA A100
Soft matching NVIDIA A100 to NVIDIA A100 PCIe
Soft matching NVIDIA A100 to NVIDIA A100 SXM4 40 GB
Found price: 24875.0

==== System: Yi-34B ====
Could not find hardware model for Yi-34B


==== System: Skywork-13B ====
Trying NVIDIA A800
Could not find hardware model for NVIDIA A800


==== System: ChatGLM3 ====
Could not find hardware model for ChatGLM3


==== System: XGen-7B ====
Trying Google TPU v4
Could not find hardware model for Google TPU v4


==== System: Falcon 180B ====
Trying NVIDIA A100 SXM4 40 GB
Found price: 24875.0

==== System: Llama 2-70B ====
Trying NVIDIA A100 SXM4 80 GB
Found price: 25641.25

==== System: Llama 2-34B ====
Trying

In [207]:
cost_df

,System,Domain,Task,Authors,Notability criteria,Notability criteria notes,Model accessibility,Link,Citations,Reference,...,Batch size notes,Organization categorization,Foundation model,Training compute lower bound,Training compute upper bound,Training chip-hours,Code accessibility,Dataset accessibility,Accessibility notes,Cost
39,Gemini Ultra,Multimodal,"Language modelling,Visual question answering,C...",Gemini Team,SOTA improvement,""" Evaluation on a broad range of benchmarks sh...",Hosted access (no API),https://storage.googleapis.com/deepmind-media/...,252.0,Gemini: A Family of Highly Capable Multimodal ...,...,NaN,Industry,NaN,NaN,NaN,132000000.0,NaN,NaN,NaN,NaN
46,Qwen-72B,Language,"Chat,Code generation","Jinze Bai, Shuai Bai, Yunfei Chu, Zeyu Cui, Ka...",SOTA improvement,"SOTA on several Chinese benchmarks, with highe...",Permissive license (depr.),https://huggingface.co/Qwen/Qwen-72B,NaN,NaN,...,Table 1 https://arxiv.org/abs/2309.16609\n(thi...,Industry,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54,Inflection-2,Language,Language modelling,NaN,Significant use,Inflection-2 either already powers Pi or soon ...,API access,https://inflection.ai/inflection-2,NaN,Inflection-2: The Next Step Up,...,NaN,Industry,checked,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59,Nemotron-3-8B,Language,"Chat,Language generation",NaN,SOTA improvement,"""The Nemotron-3-8B-QA model offers state-of-th...",Permissive license (depr.),https://developer.nvidia.com/blog/nvidia-ai-fo...,NaN,NVIDIA AI Foundation Models: Build Custom Ente...,...,NaN,Industry,NaN,NaN,NaN,NaN,NaN,NaN,NaN,441677.389916
83,Yi-34B,Language,Chat,NaN,Significant use,2nd most popular model on HuggingFace: https:/...,Permissive license (depr.),https://arxiv.org/abs/2403.04652,NaN,NaN,...,NaN,Industry,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,GNMT,Language,Translation,"Yonghui Wu, Mike Schuster, Zhifeng Chen, Quoc ...",Highly cited,NaN,NaN,https://arxiv.org/abs/1609.08144,6105.0,Google's Neural Machine Translation System: Br...,...,NaN,Industry,NaN,NaN,NaN,414720.0,NaN,NaN,NaN,NaN
1131,AlphaGo Lee,Games,Go,"David Silver, Aja Huang, Chris J. Maddison, Ar...",Highly cited,NaN,NaN,https://www.nature.com/articles/nature16961,14733.0,Mastering the game of Go with deep neural netw...,...,NaN,Industry,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1135,ResNet-152 (ImageNet),Vision,Image classification,"Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun",Highly cited,NaN,NaN,https://arxiv.org/abs/1512.03385,154061.0,Deep Residual Learning for Image Recognition,...,NaN,Industry,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1137,DeepSpeech2 (English),Speech,Speech recognition,"Dario Amodei, Rishita Anubhai, Eric Battenberg...",Highly cited,NaN,NaN,https://arxiv.org/abs/1512.02595,2741.0,Deep Speech 2: End-to-End Speech Recognition i...,...,NaN,Industry,NaN,NaN,NaN,301.0,NaN,NaN,NaN,NaN


In [208]:
cost_df['Cost'].notna().sum()

31

Use the below to check data availability for specific systems

In [209]:
# system = 'WizardLM-7B'
# row = cost_df.loc[cost_df['System'] == system]
# print('Cost:', row['Cost'].values[0])
# print('Training hardware:', row['Training hardware'].values[0])
# print('Training time (hours):', row['Training time (hours)'].values[0])
# print('Hardware quantity:', row['Hardware quantity'].values[0])
# print('Hardware utilization:', row['Hardware utilization'].values[0])

# Apply inflation adjustment

In [210]:
# TODO move to function

In [211]:
cost_df['Publication date']

39     2023-12-06
46     2023-11-30
54     2023-11-22
59     2023-11-15
83     2023-11-02
          ...    
1095   2016-09-26
1131   2016-01-27
1135   2015-12-10
1137   2015-12-08
1141   2015-10-01
Name: Publication date, Length: 148, dtype: datetime64[ns]

In [212]:
from_year_month = cost_df['Publication date'].apply(str)
cost_df['Publication date'] = from_year_month

In [213]:
cost_df['Publication date']

39      2023-12-06 00:00:00
46      2023-11-30 00:00:00
54      2023-11-22 00:00:00
59      2023-11-15 00:00:00
83      2023-11-02 00:00:00
               ...         
1095    2016-09-26 00:00:00
1131    2016-01-27 00:00:00
1135    2015-12-10 00:00:00
1137    2015-12-08 00:00:00
1141    2015-10-01 00:00:00
Name: Publication date, Length: 148, dtype: object

In [214]:
cost_df = adjust_column_for_inflation(cost_df, 'Cost', 'data/PCU518210518210.csv', '2023-12-01')

In [215]:
cost_df['Cost (inflation-adjusted)'].dropna()

59     4.427114e+05
135    1.684096e+07
174    2.118069e+06
186    2.858053e+06
188    2.023026e+05
225    3.154703e+05
256    6.216566e+05
263    5.449438e+07
264    5.286529e+05
279    9.809609e+05
334    1.072482e+06
340    1.478221e+05
375    1.066945e+06
376    3.556485e+05
380    5.125864e+05
439    8.047873e+05
443    1.442237e+05
472    2.027732e+05
495    1.259403e+05
514    1.199130e+05
542    3.469823e+06
549    6.438386e+05
582    1.625185e+04
608    1.051746e+05
656    5.218728e+04
724    3.089796e+06
769    3.061974e+03
810    1.479869e+05
835    1.095963e+05
922    3.938203e+02
932    2.607452e+03
Name: Cost (inflation-adjusted), dtype: float64

In [216]:
cost_df['Cost (inflation-adjusted)'].notna().sum()

31

# Regression

In [217]:
cost_df['Publication date (float)'] = datetime_to_float_year(pd.to_datetime(cost_df['Publication date']))

In [218]:
reg_results = fit_ols_regression(cost_df, ['Publication date (float)'], 'Cost (inflation-adjusted)', logy=True)
reg_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.490
Model:                            OLS   Adj. R-squared:                  0.472
Method:                 Least Squares   F-statistic:                     27.84
Date:                Fri, 22 Mar 2024   Prob (F-statistic):           1.18e-05
Time:                        15:23:58   Log-Likelihood:                -34.596
No. Observations:                  31   AIC:                             73.19
Df Residuals:                      29   BIC:                             76.06
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1001.9027    190.920     -5.248      0.000   -1392.378    -611.427
x1             0.4982      0.094      5.276      0.000       0.305       0.691
==============================================================================
Omnibus:                        3.824   Durbin-Watson:                   2.471
Prob(Omnibus):                  0.148   Jarque-Bera (JB):                2.855
Skew:                           0.741   Prob(JB):                        0.240
Kurtosis:                       3.109   Cond. No.                     2.81e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.81e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [219]:
print_growth_rates(reg_results)

0.50 OOMs/year (95% CI: 0.31, 0.69)
3.1x/year (95% CI: 2.0x, 4.9x)
doubling time of 7 months (95% CI: 5, 12)


In [220]:
pred_start_year = 2015
pred_end_year = 2025
pred_start_date = f'{pred_start_year}-01-01'
pred_end_date = f'{pred_end_year}-01-01'

pred_years = pd.DataFrame({'Publication date (float)': np.linspace(pred_start_year, pred_end_year, 100)})
pred_years

,Publication date (float)
0,2015.00000
1,2015.10101
2,2015.20202
3,2015.30303
4,2015.40404
...,...
95,2024.59596
96,2024.69697
97,2024.79798
98,2024.89899


In [221]:

predicted_cost_df = get_predictions(reg_results, pred_years, ['Publication date (float)'])
predicted_cost_df['Publication date'] = predicted_cost_df['Publication date (float)'].apply(float_year_to_datetime)
predicted_cost_df

,mean,mean_se,mean_ci_lower,mean_ci_upper,obs_ci_lower,obs_ci_upper,Publication date (float),Publication date
0,1.964227,0.676471,0.580689,3.347765,-0.122353,4.050808,2015.00000,2015-01-01
1,2.014550,0.667134,0.650107,3.378993,-0.059418,4.088519,2015.10101,2015-02-06
2,2.064873,0.657803,0.719514,3.410232,0.003409,4.126337,2015.20202,2015-03-15
3,2.115196,0.648479,0.788908,3.441484,0.066128,4.164264,2015.30303,2015-04-21
4,2.165519,0.639160,0.858289,3.472748,0.128735,4.202303,2015.40404,2015-05-28
...,...,...,...,...,...,...,...,...
95,6.744906,0.279595,6.173069,7.316742,5.081584,8.408227,2024.59596,2024-08-06
96,6.795228,0.287944,6.206317,7.384140,5.125960,8.464497,2024.69697,2024-09-12
97,6.845551,0.296365,6.239418,7.451685,5.170129,8.520973,2024.79798,2024-10-19
98,6.895874,0.304851,6.272384,7.519365,5.214095,8.577654,2024.89899,2024-11-25


# Plots

In [222]:
fig = px.scatter(
    cost_df,
    x='Publication date',
    y='Cost (inflation-adjusted)',
    text='System',
    log_y=True,
)
fig.update_traces(textposition='top center')

# no legend
fig.update_layout(showlegend=False)

# axis labels
fig.update_xaxes(title_text='Publication date')
fig.update_yaxes(title_text='Cost (2023 USD)')

# title
fig.update_layout(title_text='Cost of cloud compute to train large ML systems')

# update size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
)

# font size
fig.update_layout(
    font=dict(
        size=14,
    )
)

# axis limits
fig.update_xaxes(range=['2015-01-01', '2025-01-01'])

# margins
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))

save_plot(fig, results_dir, 'cost_scatter')

fig.show()

In [223]:
label_systems = ['GNMT', 'Megatron-BERT', 'GPT-3 175B (davinci)', 'PaLM (540B)', 'Llama 2-70B', 'Falcon 180B', 'GPT-4', 'Gemini Ultra']

fig = px.scatter(
    cost_df.loc[cost_df['System'].isin(label_systems)],
    x='Publication date',
    y='Cost (inflation-adjusted)',
    text='System',
    log_y=True,
)

fig.add_scatter(
    x=cost_df['Publication date'],
    y=cost_df['Cost (inflation-adjusted)'],
    mode='markers',
    showlegend=False,
)

# Marker color
fig.update_traces(
    marker=dict(
        color='rgb(0,100,200)',
    ),
    selector=dict(mode='markers'),
)

# Shade in CI
fig.add_scatter(
    x=predicted_cost_df['Publication date'],
    y=10**predicted_cost_df['mean_ci_lower'],
    mode='lines',
    line=dict(width=0),
    showlegend=False,
)
fig.add_scatter(
    x=predicted_cost_df['Publication date'],
    y=10**predicted_cost_df['mean_ci_upper'],
    mode='lines',
    fill='tonexty',
    fillcolor='rgba(0,100,200,0.2)',
    line=dict(width=0),
    name='95% CI of mean',
)
fig.add_scatter(
    x=predicted_cost_df['Publication date'],
    y=10**predicted_cost_df['mean'],
    mode='lines',
    line=dict(color='rgb(0,100,200)'),
    name=f'Regression mean (growth rate: {10**reg_results.params[1]:.1f}x per year)',
)

fig.update_traces(textposition='top center')

# legend on top
fig.update_layout(legend=dict(
    orientation='h',
    yanchor='top',
    y=-0.15,
    xanchor='center',
    x=0.5,
))

# axis labels
fig.update_xaxes(title_text='Publication date')
fig.update_yaxes(title_text='Cost (2023 USD)')

# title
fig.update_layout(title_text='Cost of cloud compute to train large ML systems')

# update size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
    title_x=0.5,
)

# font size
fig.update_layout(
    font=dict(
        size=14,
    )
)

# axis limits
fig.update_xaxes(range=[pred_start_date, pred_end_date])

# margins
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))

save_plot(fig, results_dir, 'cost_regression')

fig.show()

# Export data

In [224]:
keep_cols = [
    'System',
    'Domain',
    'Task',
    'Model accessibility',
    'Reference',
    'Publication date',
    'Organization',
    'Parameters',
    'Training compute (FLOP)',
    'Training dataset size (datapoints)',
    'Epochs',
    'Training time (hours)',
    'Training hardware',
    'Country (from Organization)',
    'Base model',
    'Finetune compute (FLOP)',
    'Hardware quantity',
    'Hardware utilization',
    'Training cloud compute vendor',
    'Training data center',
    # 'Training time (chip hours)',
    'Cost',
    'Cost (inflation-adjusted)',
]
cost_df[keep_cols]

,System,Domain,Task,Model accessibility,Reference,Publication date,Organization,Parameters,Training compute (FLOP),Training dataset size (datapoints),...,Training hardware,Country (from Organization),Base model,Finetune compute (FLOP),Hardware quantity,Hardware utilization,Training cloud compute vendor,Training data center,Cost,Cost (inflation-adjusted)
39,Gemini Ultra,Multimodal,"Language modelling,Visual question answering,C...",Hosted access (no API),Gemini: A Family of Highly Capable Multimodal ...,2023-12-06 00:00:00,Google DeepMind,NaN,5.000000e+25,NaN,...,Google TPU v4,Multinational,NaN,NaN,55000.0,NaN,NaN,NaN,NaN,NaN
46,Qwen-72B,Language,"Chat,Code generation",Permissive license (depr.),NaN,2023-11-30 00:00:00,Alibaba,7.200000e+10,1.300000e+24,NaN,...,NaN,China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54,Inflection-2,Language,Language modelling,API access,Inflection-2: The Next Step Up,2023-11-22 00:00:00,Inflection AI,NaN,1.001000e+25,NaN,...,NVIDIA H100 SXM5,United States of America,NaN,NaN,5000.0,NaN,NaN,NaN,NaN,NaN
59,Nemotron-3-8B,Language,"Chat,Language generation",Permissive license (depr.),NVIDIA AI Foundation Models: Build Custom Ente...,2023-11-15 00:00:00,NVIDIA,8.000000e+09,1.800000e+23,NaN,...,NVIDIA A100,United States of America,NaN,NaN,1024.0,0.34,NaN,NaN,441677.389916,442711.362517
83,Yi-34B,Language,Chat,Permissive license (depr.),NaN,2023-11-02 00:00:00,01.AI,3.400000e+10,6.100000e+23,NaN,...,NaN,China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,GNMT,Language,Translation,NaN,Google's Neural Machine Translation System: Br...,2016-09-26 00:00:00,Google,2.780000e+08,6.900000e+21,360000000.0,...,NVIDIA Tesla K80,United States of America,NaN,NaN,96.0,NaN,NaN,NaN,NaN,NaN
1131,AlphaGo Lee,Games,Go,NaN,Mastering the game of Go with deep neural netw...,2016-01-27 00:00:00,DeepMind,NaN,1.900000e+21,29400000.0,...,NaN,United Kingdom of Great Britain and Northern I...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1135,ResNet-152 (ImageNet),Vision,Image classification,NaN,Deep Residual Learning for Image Recognition,2015-12-10 00:00:00,Microsoft,6.000000e+07,1.210000e+19,1280000.0,...,NaN,United States of America,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1137,DeepSpeech2 (English),Speech,Speech recognition,NaN,Deep Speech 2: End-to-End Speech Recognition i...,2015-12-08 00:00:00,Baidu Research - Silicon Valley AI Lab,3.800000e+07,2.600000e+19,163339200.0,...,NVIDIA GTX Titan X,United States of America,NaN,NaN,NaN,0.45,NaN,NaN,NaN,NaN


In [225]:
cost_df[keep_cols].to_csv(results_dir + 'price dataset.csv', index=False)